In [1]:
import os
import time
import io
import hashlib
import numpy as np
from PIL import Image
from utils import parseBBOXFile, getTagValue

In [2]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
import tensorflow as tf
from object_detection.utils import dataset_util, label_map_util

In [4]:
label_map = label_map_util.load_labelmap('../annotations/label_map.pbtxt')
label_map_dict = label_map_util.get_label_map_dict(label_map)
label_map_dict

{'car': 1, 'truck': 2, 'bus': 3, 'bike': 4, 'person': 5}

Convert a folder of images + xml files to TF-record format, needed for training.

In [7]:
def getFiles(path):
    files = []
    for file in os.listdir(path):
        with open('{}/{}'.format(path, file), 'r') as f:
            files += [item.strip().lower() for item in f]
    return files

In [8]:
def convertFolder(name, image_path, annotate_path, image_format = b'jpg'):
    files = getFiles(image_path)
    if len(files) == 0:
        return
    
    print("Creating TF-record:", '{}/{}.record'.format(annotate_path, name))
    with tf.io.TFRecordWriter('{}/{}.record'.format(annotate_path, name)) as writer:
        for f in files:
            img, xml = '../images/{}.jpg'.format(f), '../images/{}.xml'.format(f)
            encoded_jpg = None
            with tf.io.gfile.GFile(img, 'rb') as fid:
                encoded_jpg = fid.read()

            image = Image.open(io.BytesIO(encoded_jpg))
            key = hashlib.sha256(encoded_jpg).hexdigest().encode('utf8')
            width, height = image.size
            filename = '{}.jpg'.format(f.split('/')[-1]).encode('utf8')
            xmins, xmaxs = [], []
            ymins, ymaxs = [], []
            classes_name, classes_id = [], []
            truncated = []
            poses = []
            difficult = []

            for obj in parseBBOXFile(xml):
                lbl = getTagValue(obj, 'name')

                xmin = (float(getTagValue(obj, 'xmin')) / float(width))
                xmax = (float(getTagValue(obj, 'xmax')) / float(width))

                ymin = (float(getTagValue(obj, 'ymin')) / float(height))
                ymax = (float(getTagValue(obj, 'ymax')) / float(height))

                classes_name.append(lbl.encode('utf8'))
                classes_id.append(label_map_dict[lbl])
                xmins.append(xmin)
                xmaxs.append(xmax)
                ymins.append(ymin)
                ymaxs.append(ymax)
                
                truncated.append(int(getTagValue(obj, 'truncated')))
                difficult.append(int(getTagValue(obj, 'difficult')))
                poses.append(getTagValue(obj, 'pose').encode('utf8'))

            record = tf.train.Example(features=tf.train.Features(feature={
                'image/height': dataset_util.int64_feature(height),
                'image/width': dataset_util.int64_feature(width),
                'image/filename': dataset_util.bytes_feature(filename),
                'image/source_id': dataset_util.bytes_feature(filename),
                'image/key/sha256': dataset_util.bytes_feature(key),
                'image/encoded': dataset_util.bytes_feature(encoded_jpg),
                'image/format': dataset_util.bytes_feature(image_format),
                'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
                'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
                'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
                'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
                'image/object/class/text': dataset_util.bytes_list_feature(classes_name),
                'image/object/class/label': dataset_util.int64_list_feature(classes_id),
                'image/object/difficult': dataset_util.int64_list_feature(difficult),
                'image/object/truncated': dataset_util.int64_list_feature(truncated),
                'image/object/view': dataset_util.bytes_list_feature(poses),
            })).SerializeToString()
            writer.write(record)
            print("Wrote", f, 'to the record.')
            #break
    print("Finished!")

In [10]:
convertFolder('train', '../images/train', '../annotations')

Creating TF-record: ../annotations/train.record
Wrote dark/2015_01879 to the record.
Wrote dark/2015_01880 to the record.
Wrote dark/2015_01881 to the record.
Wrote dark/2015_01882 to the record.
Wrote dark/2015_01883 to the record.
Wrote dark/2015_01884 to the record.
Wrote dark/2015_01885 to the record.
Wrote dark/2015_01886 to the record.
Wrote dark/2015_01889 to the record.
Wrote dark/2015_01890 to the record.
Wrote dark/2015_01891 to the record.
Wrote dark/2015_01892 to the record.
Wrote dark/2015_01893 to the record.
Wrote dark/2015_01894 to the record.
Wrote dark/2015_01896 to the record.
Wrote dark/2015_01897 to the record.
Wrote dark/2015_01898 to the record.
Wrote dark/2015_01899 to the record.
Wrote dark/2015_01900 to the record.
Wrote dark/2015_01903 to the record.
Wrote dark/2015_01905 to the record.
Wrote dark/2015_01906 to the record.
Wrote dark/2015_01907 to the record.
Wrote dark/2015_01908 to the record.
Wrote dark/2015_01909 to the record.
Wrote dark/2015_01910 to th

Wrote dark/2015_02149 to the record.
Wrote dark/2015_02150 to the record.
Wrote dark/2015_02151 to the record.
Wrote dark/2015_02153 to the record.
Wrote dark/2015_02154 to the record.
Wrote dark/2015_02155 to the record.
Wrote dark/2015_02156 to the record.
Wrote dark/2015_02157 to the record.
Wrote dark/2015_02158 to the record.
Wrote dark/2015_02159 to the record.
Wrote dark/2015_02160 to the record.
Wrote dark/2015_02161 to the record.
Wrote dark/2015_02163 to the record.
Wrote dark/2015_02164 to the record.
Wrote dark/2015_02165 to the record.
Wrote dark/2015_02166 to the record.
Wrote dark/2015_02167 to the record.
Wrote dark/2015_02168 to the record.
Wrote dark/2015_02169 to the record.
Wrote dark/2015_02170 to the record.
Wrote dark/2015_02171 to the record.
Wrote dark/2015_02172 to the record.
Wrote dark/2015_02173 to the record.
Wrote dark/2015_02174 to the record.
Wrote dark/2015_02175 to the record.
Wrote dark/2015_02176 to the record.
Wrote dark/2015_02177 to the record.
W

Wrote dark/2015_02398 to the record.
Wrote dark/2015_02399 to the record.
Wrote dark/2015_02400 to the record.
Wrote dark/2015_02401 to the record.
Wrote dark/2015_02402 to the record.
Wrote dark/2015_02403 to the record.
Wrote dark/2015_02404 to the record.
Wrote dark/2015_02405 to the record.
Wrote dark/2015_02406 to the record.
Wrote dark/2015_02407 to the record.
Wrote dark/2015_02408 to the record.
Wrote dark/2015_02409 to the record.
Wrote dark/2015_02410 to the record.
Wrote dark/2015_02411 to the record.
Wrote dark/2015_02412 to the record.
Wrote dark/2015_02413 to the record.
Wrote dark/2015_02414 to the record.
Wrote dark/2015_02415 to the record.
Wrote dark/2015_02416 to the record.
Wrote dark/2015_02417 to the record.
Wrote dark/2015_02418 to the record.
Wrote dark/2015_02419 to the record.
Wrote dark/2015_02420 to the record.
Wrote dark/2015_02421 to the record.
Wrote dark/2015_02422 to the record.
Wrote dark/2015_02423 to the record.
Wrote dark/2015_02424 to the record.
W

Wrote dark/2015_02654 to the record.
Wrote dark/2015_02655 to the record.
Wrote dark/2015_02656 to the record.
Wrote dark/2015_02657 to the record.
Wrote dark/2015_02659 to the record.
Wrote dark/2015_02660 to the record.
Wrote dark/2015_02661 to the record.
Wrote dark/2015_02662 to the record.
Wrote dark/2015_02663 to the record.
Wrote dark/2015_02664 to the record.
Wrote dark/2015_02665 to the record.
Wrote dark/2015_02666 to the record.
Wrote dark/2015_02667 to the record.
Wrote dark/2015_02668 to the record.
Wrote dark/2015_02669 to the record.
Wrote dark/2015_02670 to the record.
Wrote dark/2015_02671 to the record.
Wrote dark/2015_02672 to the record.
Wrote dark/2015_02673 to the record.
Wrote dark/2015_02674 to the record.
Wrote dark/2015_02675 to the record.
Wrote dark/2015_02676 to the record.
Wrote dark/2015_02677 to the record.
Wrote dark/2015_02679 to the record.
Wrote dark/2015_02680 to the record.
Wrote dark/2015_02681 to the record.
Wrote dark/2015_02682 to the record.
W

Wrote dark/2015_02901 to the record.
Wrote dark/2015_02902 to the record.
Wrote dark/2015_02903 to the record.
Wrote dark/2015_02904 to the record.
Wrote dark/2015_02905 to the record.
Wrote dark/2015_02906 to the record.
Wrote dark/2015_02907 to the record.
Wrote dark/2015_02908 to the record.
Wrote dark/2015_02909 to the record.
Wrote dark/2015_02910 to the record.
Wrote dark/2015_02911 to the record.
Wrote dark/2015_02913 to the record.
Wrote dark/2015_02914 to the record.
Wrote dark/2015_02915 to the record.
Wrote dark/2015_02916 to the record.
Wrote dark/2015_02917 to the record.
Wrote dark/2015_02919 to the record.
Wrote dark/2015_02920 to the record.
Wrote dark/2015_02921 to the record.
Wrote dark/2015_02922 to the record.
Wrote dark/2015_02923 to the record.
Wrote dark/2015_02924 to the record.
Wrote dark/2015_02925 to the record.
Wrote dark/2015_02926 to the record.
Wrote dark/2015_02927 to the record.
Wrote dark/2015_02928 to the record.
Wrote dark/2015_02929 to the record.
W

...

Wrote dark/2015_06680 to the record.
Wrote dark/2015_06681 to the record.
Wrote dark/2015_06682 to the record.
Wrote dark/2015_06684 to the record.
Wrote dark/2015_06685 to the record.
Wrote dark/2015_06686 to the record.
Wrote dark/2015_06687 to the record.
Wrote dark/2015_06688 to the record.
Wrote dark/2015_06689 to the record.
Wrote dark/2015_06691 to the record.
Wrote dark/2015_06693 to the record.
Wrote dark/2015_06694 to the record.
Wrote dark/2015_06695 to the record.
Wrote dark/2015_06696 to the record.
Wrote dark/2015_06697 to the record.
Wrote dark/2015_06698 to the record.
Wrote dark/2015_06699 to the record.
Wrote dark/2015_06700 to the record.
Wrote dark/2015_06701 to the record.
Wrote dark/2015_06702 to the record.
Wrote dark/2015_06703 to the record.
Wrote dark/2015_06704 to the record.
Wrote dark/2015_06705 to the record.
Wrote dark/2015_06707 to the record.
Wrote dark/2015_06708 to the record.
Wrote dark/2015_06709 to the record.
Wrote dark/2015_06710 to the record.
W

Wrote raw/e39_aug_n_06-01-23-39 to the record.
Wrote raw/e39_aug_n_06-01-23-40 to the record.
Wrote raw/e39_aug_n_06-01-23-50 to the record.
Wrote raw/e39_aug_n_06-01-23-56 to the record.
Wrote raw/e39_aug_n_07-01-00-01 to the record.
Wrote raw/e39_aug_n_07-01-00-07 to the record.
Wrote raw/e39_aug_n_07-01-00-12 to the record.
Wrote raw/e39_aug_n_07-01-00-14 to the record.
Wrote raw/e39_aug_n_07-01-11-10 to the record.
Wrote raw/e39_aug_n_07-01-11-14 to the record.
Wrote raw/e39_aug_n_07-01-11-20 to the record.
Wrote raw/e39_aug_n_07-01-11-25 to the record.
Wrote raw/e39_aug_n_07-01-11-32 to the record.
Wrote raw/e39_aug_n_07-01-11-40 to the record.
Wrote raw/e39_aug_n_07-01-11-49 to the record.
Wrote raw/e39_aug_n_07-01-11-55 to the record.
Wrote raw/e39_aug_n_07-01-12-00 to the record.
Wrote raw/e39_aug_n_07-01-12-05 to the record.
Wrote raw/e39_aug_n_07-01-12-08 to the record.
Wrote raw/e39_aug_n_07-01-12-15 to the record.
Wrote raw/e39_aug_n_07-01-12-21 to the record.
Wrote raw/e39

Wrote raw/e39_aug_n_08-01-15-29 to the record.
Wrote raw/e39_aug_n_08-01-15-34 to the record.
Wrote raw/e39_aug_n_08-01-15-39 to the record.
Wrote raw/e39_aug_n_08-01-15-42 to the record.
Wrote raw/e39_aug_n_08-01-15-48 to the record.
Wrote raw/e39_aug_n_08-01-15-50 to the record.
Wrote raw/e39_aug_n_08-01-15-55 to the record.
Wrote raw/e39_aug_n_08-01-16-00 to the record.
Wrote raw/e39_aug_n_08-01-16-05 to the record.
Wrote raw/e39_aug_n_08-01-16-06 to the record.
Wrote raw/e39_aug_n_08-01-16-12 to the record.
Wrote raw/e39_aug_n_08-01-16-23 to the record.
Wrote raw/e39_aug_n_08-01-16-27 to the record.
Wrote raw/e39_aug_n_08-01-16-32 to the record.
Wrote raw/e39_aug_n_08-01-16-40 to the record.
Wrote raw/e39_aug_n_08-01-16-47 to the record.
Wrote raw/e39_aug_n_08-01-16-49 to the record.
Wrote raw/e39_aug_n_08-01-16-54 to the record.
Wrote raw/e39_aug_n_08-01-17-00 to the record.
Wrote raw/e39_aug_n_08-01-17-17 to the record.
Wrote raw/e39_aug_n_08-01-17-24 to the record.
Wrote raw/e39

Wrote raw/e39_aug_s_07-01-15-33 to the record.
Wrote raw/e39_aug_s_07-01-15-38 to the record.
Wrote raw/e39_aug_s_07-01-15-48 to the record.
Wrote raw/e39_aug_s_07-01-15-54 to the record.
Wrote raw/e39_aug_s_07-01-16-00 to the record.
Wrote raw/e39_aug_s_07-01-16-05 to the record.
Wrote raw/e39_aug_s_07-01-16-11 to the record.
Wrote raw/e39_aug_s_07-01-16-17 to the record.
Wrote raw/e39_aug_s_07-01-16-22 to the record.
Wrote raw/e39_aug_s_07-01-16-34 to the record.
Wrote raw/e39_aug_s_07-01-16-39 to the record.
Wrote raw/e39_aug_s_07-01-16-42 to the record.
Wrote raw/e39_aug_s_07-01-16-49 to the record.
Wrote raw/e39_aug_s_07-01-16-54 to the record.
Wrote raw/e39_aug_s_07-01-17-00 to the record.
Wrote raw/e39_aug_s_07-01-17-05 to the record.
Wrote raw/e39_aug_s_07-01-17-08 to the record.
Wrote raw/e39_aug_s_07-01-17-12 to the record.
Wrote raw/e39_aug_s_07-01-17-19 to the record.
Wrote raw/e39_aug_s_07-01-17-23 to the record.
Wrote raw/e39_aug_s_07-01-17-24 to the record.
Wrote raw/e39

Wrote raw/e39_aug_s_08-01-22-33 to the record.
Wrote raw/e39_aug_s_08-01-22-40 to the record.
Wrote raw/e39_aug_s_08-01-22-47 to the record.
Wrote raw/e39_aug_s_08-01-22-58 to the record.
Wrote raw/e39_aug_s_18-01-14-42 to the record.
Wrote raw/e39_aug_s_18-01-14-48 to the record.
Wrote raw/e39_aug_s_18-01-14-59 to the record.
Wrote raw/e39_aug_s_18-01-16-23 to the record.
Wrote raw/e39_aug_s_18-01-23-16 to the record.
Wrote raw/e39_aug_s_18-01-23-24 to the record.
Wrote raw/e39_aug_s_18-01-23-51 to the record.
Wrote raw/e39_aug_s_19-01-13-01 to the record.
Wrote raw/e39_aug_s_19-01-15-20 to the record.
Wrote raw/e39_vatland_09-01-12-33 to the record.
Wrote raw/e39_vatland_09-01-18-40 to the record.
Wrote raw/e39_vatland_10-01-16-07 to the record.
Wrote raw/e39_vatland_10-01-16-40 to the record.
Wrote raw/e39_vatland_10-01-17-19 to the record.
Wrote raw/e39_vatland_10-01-18-40 to the record.
Wrote raw/e6_nostvet_11-01-14-57 to the record.
Wrote raw/e6_nostvet_11-01-15-07 to the record.

In [11]:
convertFolder('test', '../images/test', '../annotations')

Creating TF-record: ../annotations/test.record
Wrote dark/2015_01887 to the record.
Wrote dark/2015_01888 to the record.
Wrote dark/2015_01895 to the record.
Wrote dark/2015_01901 to the record.
Wrote dark/2015_01902 to the record.
Wrote dark/2015_01904 to the record.
Wrote dark/2015_01924 to the record.
Wrote dark/2015_01926 to the record.
Wrote dark/2015_01927 to the record.
Wrote dark/2015_01929 to the record.
Wrote dark/2015_01937 to the record.
Wrote dark/2015_01967 to the record.
Wrote dark/2015_01972 to the record.
Wrote dark/2015_01989 to the record.
Wrote dark/2015_01992 to the record.
Wrote dark/2015_02011 to the record.
Wrote dark/2015_02029 to the record.
Wrote dark/2015_02030 to the record.
Wrote dark/2015_02033 to the record.
Wrote dark/2015_02053 to the record.
Wrote dark/2015_02057 to the record.
Wrote dark/2015_02079 to the record.
Wrote dark/2015_02080 to the record.
Wrote dark/2015_02100 to the record.
Wrote dark/2015_02107 to the record.
Wrote dark/2015_02116 to the

Wrote raw/e39_aug_n_06-01-17-39 to the record.
Wrote raw/e39_aug_n_06-01-18-54 to the record.
Wrote raw/e39_aug_n_06-01-20-01 to the record.
Wrote raw/e39_aug_n_06-01-20-06 to the record.
Wrote raw/e39_aug_n_06-01-20-54 to the record.
Wrote raw/e39_aug_n_06-01-22-13 to the record.
Wrote raw/e39_aug_n_07-01-12-54 to the record.
Wrote raw/e39_aug_n_07-01-14-31 to the record.
Wrote raw/e39_aug_n_07-01-18-31 to the record.
Wrote raw/e39_aug_n_07-01-19-50 to the record.
Wrote raw/e39_aug_n_07-01-20-15 to the record.
Wrote raw/e39_aug_n_07-01-20-26 to the record.
Wrote raw/e39_aug_n_07-01-22-32 to the record.
Wrote raw/e39_aug_n_07-01-23-00 to the record.
Wrote raw/e39_aug_n_08-01-00-19 to the record.
Wrote raw/e39_aug_n_08-01-00-40 to the record.
Wrote raw/e39_aug_n_08-01-11-20 to the record.
Wrote raw/e39_aug_n_08-01-13-22 to the record.
Wrote raw/e39_aug_n_08-01-14-07 to the record.
Wrote raw/e39_aug_n_08-01-14-33 to the record.
Wrote raw/e39_aug_n_08-01-14-50 to the record.
Wrote raw/e39

In [27]:
#for record in tf.data.TFRecordDataset("../annotations/test.record"):
#    print(tf.train.SequenceExample.FromString(record.numpy()))